In [63]:
import os
import astropy
import numpy as np
from astropy.table import Table
from astropy.table import Column
import glob
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from mpl_toolkits.mplot3d import Axes3D
import copy

In [3]:
sample = Table.read("/home/xhall/Documents/NewZTF/ML_sample.ascii", format = "ascii")
sample.rename_column('col1', 'ZTF_Name')
sample.rename_column('col2', "Class")
sample.rename_column('col8', "Version")

In [64]:
snidoutput = np.load("//home/xhall/Documents/NewZTF/sample_2020/SNID_2020_output.npy",allow_pickle=True)

In [66]:
ZTable_best = Table(
                    names=("Version", "ZTF_Name",
                           "z_sntemplate", "z_rlap", "z_snid", "z_snid_err", "z_level"
                           , "rank_1", "sntemplate_1", "rlap_1", "c_snid_1", "z_snid_1", "z_snid_err_1", "age_1", "age_flag_1"
                           , "rank_2", "sntemplate_2", "rlap_2", "c_snid_2", "z_snid_2", "z_snid_err_2", "age_2", "age_flag_2"
                           , "rank_3", "sntemplate_3", "rlap_3", "c_snid_3", "z_snid_3", "z_snid_err_3", "age_3", "age_flag_3"
                           , "rank_4", "sntemplate_4", "rlap_4", "c_snid_4", "z_snid_4", "z_snid_err_4", "age_4", "age_flag_4"
                           , "rank_5", "sntemplate_5", "rlap_5", "c_snid_5", "z_snid_5", "z_snid_err_5", "age_5", "age_flag_5"
                    ),
                    meta={"name": "Redscore Results"},
                    dtype=("U64", "U64",
                           "U64", "float64", "float64", "float64", "int32",
                            "int32", "U64", "float64", "U64", "float64", "float64", "float64", "int32",
                            "int32", "U64", "float64", "U64", "float64", "float64", "float64", "int32",
                            "int32", "U64", "float64", "U64", "float64", "float64", "float64", "int32",
                            "int32", "U64", "float64", "U64", "float64", "float64", "float64", "int32",
                            "int32", "U64", "float64", "U64", "float64", "float64", "float64", "int32",
                          )
                    )
len(["Version", "ZTF_Name",
                           "z_sntemplate", "z_rlap", "z_snid", "z_snid_err", "age", "age_flag", "z_level"
                           , "rank_1", "sntemplate_1", "rlap_1", "c_snid_1", "z_snid_1", "z_snid_err_1", "age_1", "age_flag_1"
                           , "rank_2", "sntemplate_2", "rlap_2", "c_snid_2", "z_snid_2", "z_snid_err_2", "age_2", "age_flag_2"
                           , "rank_3", "sntemplate_3", "rlap_3", "c_snid_3", "z_snid_3", "z_snid_err_3", "age_3", "age_flag_3"
                           , "rank_4", "sntemplate_4", "rlap_4", "c_snid_4", "z_snid_4", "z_snid_err_4", "age_4", "age_flag_4"
                           , "rank_5", "sntemplate_5", "rlap_5", "c_snid_5", "z_snid_5", "z_snid_err_5", "age_5", "age_flag_5"])

49

In [82]:
j = snidoutput[1]
Top50 = copy.deepcopy(j[1][0:50])
Top5 = copy.deepcopy(Top50[0:5])
types = np.unique(Top50["type"])
top5Types = np.unique(Top5["type"])

row = []
row.append(j[0] + ".ascii")
row.append(j[0].split("_")[0])

good = j[1][np.where(j[1]["grade"] == "good")]
if("SLSN" in str(j[1][0]["type"])):
    good = good[np.where(good["z"] <= .5)]
else:
    good = good[np.where(good["z"] <= .2)]
if(len(good) != 0):
    row.append(good[0]["sn"])
    row.append(good[0]["rlap"])
    row.append(float(good[0]["z"]))
    row.append(float(good[0]["zerr"]))
    row.append(1)
else:
    row.append(j[1][0]["sn"])
    row.append(j[1][0]["rlap"])
    row.append(float(j[1][0]["z"]))
    row.append(float(j[1][0]["zerr"]))
    row.append(0)

if(len(top5Types) >= 3):
    for i in Top5:
        row.append(i["no"])
        row.append(i["sn"])
        row.append(i["rlap"])
        row.append(i["type"])
        row.append(i["z"])
        row.append(i["zerr"])
        row.append(i["age"])
        row.append(i["age_flag"])
elif(len(top5Types) == 2):
    if(len(types) == 2):
        for i in Top5:
            row.append(i["no"])
            row.append(i["sn"])
            row.append(i["rlap"])
            row.append(i["type"])
            row.append(i["z"])
            row.append(i["zerr"])
            row.append(i["age"])
            row.append(i["age_flag"])
    else:
        newType = -1
        for i in range(len(Top50)):
            line = Top50[i]
            if(line["type"] not in top5Types):
                newType = i
                break
        for i in range(len(Top5)-1,-1,-1):
            line = Top5[i]
            unique = list(Top5["type"]).count(line["type"])
            if(unique > 1):
                Top5[i] = Top50[newType]
                break
        Top5.sort("no")
        for i in Top5:
            row.append(i["no"])
            row.append(i["sn"])
            row.append(i["rlap"])
            row.append(i["type"])
            row.append(i["z"])
            row.append(i["zerr"])
            row.append(i["age"])
            row.append(i["age_flag"])
elif(len(top5Types) == 1):
    if(len(types) == 1):
        for i in Top5:
            row.append(i["no"])
            row.append(i["sn"])
            row.append(i["rlap"])
            row.append(i["type"])
            row.append(i["z"])
            row.append(i["zerr"])
            row.append(i["age"])
            row.append(i["age_flag"])
    else:
        for k in range(3,5):
            top5Types = np.unique(Top5["type"])
            for i in Top50:
                if(i["type"] not in top5Types):
                    Top5[k] = i
                    break
        for i in Top5:
            row.append(i["no"])
            row.append(i["sn"])
            row.append(i["rlap"])
            row.append(i["type"])
            row.append(i["z"])
            row.append(i["zerr"])
            row.append(i["age"])
            row.append(i["age_flag"])
ZTable_best.add_row(row)

ValueError: Mismatch between number of vals and columns

In [68]:
ZTable_best

Version,ZTF_Name,z_sntemplate,z_rlap,z_snid,z_snid_err,z_level,rank_1,sntemplate_1,rlap_1,c_snid_1,z_snid_1,z_snid_err_1,age_1,age_flag_1,rank_2,sntemplate_2,rlap_2,c_snid_2,z_snid_2,z_snid_err_2,age_2,age_flag_2,rank_3,sntemplate_3,rlap_3,c_snid_3,z_snid_3,z_snid_err_3,age_3,age_flag_3,rank_4,sntemplate_4,rlap_4,c_snid_4,z_snid_4,z_snid_err_4,age_4,age_flag_4,rank_5,sntemplate_5,rlap_5,c_snid_5,z_snid_5,z_snid_err_5,age_5,age_flag_5
str64,str64,str64,float64,float64,float64,int32,int32,str64,float64,str64,float64,float64,float64,int32,int32,str64,float64,str64,float64,float64,float64,int32,int32,str64,float64,str64,float64,float64,float64,int32,int32,str64,float64,str64,float64,float64,float64,int32,int32,str64,float64,str64,float64,float64,float64,int32
ZTF17aaajnki_20180320_P60_v1.ascii,ZTF17aaajnki,kcSB2,17.99,-0.0,0.0015,1,1,kcSB2,17.99,Gal,-0.0,0.0015,-99.9,2,2,kcSB1,17.3,Gal,0.0001,0.0016,-99.9,2,3,PS1-10bzj,10.61,Ic-SLSN,0.0003,0.0026,6.55,0,4,PS1-10bzj,7.03,Ic-SLSN,0.0009,0.0036,15.64,0,8,sn07it,5.28,II-norm,0.0007,0.0051,191.0,0


In [79]:
for i in range(3,5,1):
    print(i)

3
4


In [47]:
test = [1,2,3]

In [50]:
4 not in test

True

In [77]:
Top5[0:3]

no,sn,type,lap,rlap,z,zerr,age,age_flag,grade
int64,str12,str7,float64,float64,float64,float64,float64,int64,str4
1,kcSB2,Gal,0.8786,17.99,-0.0,0.0015,-99.9,2,good
2,kcSB1,Gal,0.8786,17.3,0.0001,0.0016,-99.9,2,good
3,PS1-10bzj,Ic-SLSN,0.4048,10.61,0.0003,0.0026,6.55,0,good


In [75]:
Top5.sort("no")

In [73]:
Top5.reverse()

In [76]:
Top5

no,sn,type,lap,rlap,z,zerr,age,age_flag,grade
int64,str12,str7,float64,float64,float64,float64,float64,int64,str4
1,kcSB2,Gal,0.8786,17.99,-0.0,0.0015,-99.9,2,good
2,kcSB1,Gal,0.8786,17.3,0.0001,0.0016,-99.9,2,good
3,PS1-10bzj,Ic-SLSN,0.4048,10.61,0.0003,0.0026,6.55,0,good
4,PS1-10bzj,Ic-SLSN,0.4792,7.03,0.0009,0.0036,15.64,0,good
8,sn07it,II-norm,0.5551,5.28,0.0007,0.0051,191.0,0,good


In [81]:
snidoutput[1]

array(['SN2017iuk_20181204_Keck1_v1',
       <Table length=6438>
  no     sn      type    lap     rlap     z      zerr    age   age_flag grade
int64  str12     str7  float64 float64 float64 float64 float64  int64    str4
----- -------- ------- ------- ------- ------- ------- ------- -------- -----
    1    kcSB5     Gal  1.1697    5.66  0.0378  0.0028   -99.9        2  good
    2    kcSB1     Gal  1.1697    5.43   0.037  0.0028   -99.9        2  good
    3    kcSB2     Gal  1.1697    5.11  0.0369   0.003   -99.9        2  good
    4    kcSB6     Gal  1.1697    4.83   0.037   0.003   -99.9        2  good
    5    kcSB4     Gal  1.1697    4.81  0.0372  0.0031   -99.9        2  good
    6     kcSc     Gal  0.9409    4.79  0.0373  0.0031   -99.9        2  good
    7   sn98bw   Ic-BL  1.0275    4.35  0.0094  0.0179    15.7        0   bad
    8 sn1998bw   Ic-BL  1.0275    4.35  0.0094  0.0177    15.7        0   bad
    9     kcS0     Gal  1.1697     4.3   0.694  0.0046   -99.9        2   bad